# Variables and data types in python 

## Libraries and settings

In [ ]:
# Libraries
import os
import random
import numpy as np
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

## Creating an Excel file containing simulated chicken data
(yes, this is not neccessary because we have Python, but it shows how we can work with Excel spread sheets from within Python)

In [ ]:
# Empty list to store the data
col_01 = []
col_02 = []
col_03 = []
col_04 = []

# List with chicken breeds
breed = ['ISA Brown',
         'Plymouth Rock',
         'Barnevelder',
         'Australorp',
         'New Hampshire Red']

# Fill the empty lists with data
random.seed(10)
for i in range(1, 101):
    col_01.append(i)
    col_02.append(float(np.random.normal(2500, 250, 1)))
    col_03.append(breed[random.randint(0, 4)])
    col_04.append(random.randint(150, 365))

# Create a data frame from the lists
df = pd.DataFrame.from_dict({'chicken_id': col_01,
                             'weight': col_02,
                             'breed': col_03,
                             'eggs_per_year': col_04})

################################################################################

# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter('./data/chicken_data.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object
df.to_excel(writer, sheet_name='Sheet1', startrow=1, header=False, index=False)

# Get the xlsxwriter objects from the dataframe writer object
workbook = writer.book
worksheet = writer.sheets['Sheet1']

# Get the dimensions of the dataframe
(max_row, max_col) = df.shape

# Create a list of column headers, to use in add_table()
column_settings = [{'header': column} for column in df.columns]

# Add the Excel table structure. Pandas will add the data
worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

# Make the columns wider for clarity
worksheet.set_column(0, max_col - 1, 20)

################################################################################

# Create pivot table
table = pd.pivot_table(df[['breed', 'eggs_per_year']],
                       index=['breed'],
                       values=['eggs_per_year'],
                       aggfunc=np.mean)

table.to_excel(writer,
               sheet_name='Sheet2',
               startrow=0,
               header=True,
               index=True)

# Include barchart
chart = workbook.add_chart({'type': 'bar', 'subtype': 'stacked'})

# Configure the first series.
chart.add_series({
    'name':       '=Sheet2!$B$1',
    'categories': '=Sheet2!$A$2:$A$6',
    'values':     '=Sheet2!$B$2:$B$6',
})

# Add a chart title and some axis labels.
chart.set_title({'name': 'Average eggs per breed'})
chart.set_x_axis({'name': ''})
chart.set_y_axis({'name': ''})

# Set an Excel chart style.
chart.set_style(13)

# Insert the chart into the worksheet (with an offset).
worksheet.insert_chart('F1', chart, {'x_offset': 25, 'y_offset': 10})

# Insert .png - file
worksheet.insert_image('H17', './data/chickens.jpg')

# Close the Pandas Excel writer and output the Excel file.
writer.close()

## Read data from Excel-File

In [ ]:
# Read xlsx file
df = pd.read_excel('./data/chicken_data.xlsx', sheet_name='Sheet1')
df

## Explore data types

In [ ]:
# Explore data types (note that 'object' means categorical in pandas)
df.dtypes

## Change data type of variable 'weight'

In [ ]:
# Change data type (note that with .astype('int32'), the values are simply cutted!)
df['weight_new'] =  df['weight'].astype('int32')
print(df.dtypes, '\n')

df

## Create new variable 'breed_str' which has string as the data type

In [ ]:
df['breed_str'] = pd.Series(['breed'], dtype="string")
df.dtypes

## Create new categorical variable from 'eggs_per_year'

In [ ]:
# Define labels for categories
labels = ['0 - 99', '100 - 199', '20 - 365']

# Create categories
df["eggs_cat"] = pd.cut(df['eggs_per_year'], bins=[0, 100, 200, 365], labels=labels)
df[['eggs_per_year', 'eggs_cat']].head(10)

## Create new numerical variable from 'weight'

In [ ]:
# Create new variable
df['weight_kg'] = round(df['weight'] / 1000, 4)

# Show values
df

## Transform categorical variable 'breed' to matrix with binary (0/1) values

In [ ]:
# Use the get_dummies() method from pandas for conversion
df_02 = pd.get_dummies(df, drop_first=False, columns=['breed'], dtype='int')
df_02


## Show data types of all generated variables

In [ ]:
# Show data types
df_02.dtypes

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')